### Import packages

In [1]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
import os
import pyLDAvis
import pyLDAvis.gensim as gensimvis
import numpy as np
import tqdm
from tqdm import tqdm
import re
from pprint import pprint
import nltk
from nltk.corpus import stopwords
import spacy
import gensim.corpora as corpora
from pprint import pprint
from gensim.models import CoherenceModel
pd.set_option('display.max_rows', None)

C:\Users\15138\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
kjv = pd.read_csv('t_kjv.csv')

In [3]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [4]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [5]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = gensim.models.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [6]:
def _get_best_num_topics(topics_range, alpha_lst, beta_lst, corpus):
    corpus_sets = [corpus]
    # maximum coherence score (value)
    max_cv = 0.0
    alpha = 0.0
    beta = 0.0
    num_topics = 2

    # iterate through the corus
    for i in tqdm(range(len(corpus_sets))):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha_lst:
                # iterare through beta values
                for b in beta_lst:
                    # compute coherence value
                    cv = compute_coherence_values(corpus=corpus_sets[i], 
                                                dictionary=id2word, k=k, a=a, b=b)
                    if cv > max_cv:
                        max_cv = cv
                        alpha = a
                        beta = b
                        num_topics = k

    return alpha, beta, num_topics, max_cv

# KJV

## Books

### Preprocessing

In [14]:
# Create list of book documents and put them in a dataframe
booksList = []
for book in kjv.groupby('b')['t']:
    booksList.append(' '.join(book[1].tolist()))
books = pd.DataFrame(booksList, columns =['t'])

In [15]:
# Remove punctuation
books['t_processed'] = books['t'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
books['t_processed'] = books['t_processed'].map(lambda x: x.lower())

#### Tokenize words and further clean-up text

data = books.t_processed.values.tolist()
data_words = list(sent_to_words(data))

#### Creating Bigram and Trigram Models

# Build the bigram and trigram models

bigram = gensim.models.Phrases(data_words, min_count=3, threshold=5)
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#### Remove Stopwords, Make Bigrams and Lemmatize

#list of stopwords
with open('stop_words_lst.txt') as f:
    standard_stop_words = f.read().splitlines()
with open('bible_stop_words_lst.txt') as f:
    bible_stop_words = f.read().splitlines()
stop_words = standard_stop_words + bible_stop_words

### Call the functions above

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_lg", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

### Data transformation: Corpus and Dictionary

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

<>:2: DeprecationWarning: invalid escape sequence \.
<>:2: DeprecationWarning: invalid escape sequence \.
C:\Users\15138\AppData\Local\Temp/ipykernel_8720/258337712.py:2: DeprecationWarning: invalid escape sequence \.
  books['t_processed'] = books['t'].map(lambda x: re.sub('[,\.!?]', '', x))


### Model Selection

In [10]:
# Topics range
min_topics = 2
max_topics = 66
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

# List of alpha parameters
alpha_lst = ['auto']

# List of beta parameters
beta_lst = list(np.arange(0.01, 3, 0.1))

# Call to the method above
alpha, beta, num_topics, max_cv = _get_best_num_topics(topics_range, alpha_lst, beta_lst, corpus)

print("alpha: ", alpha)
print("beta: ", beta)
print("Number of topics: ", num_topics)
print("Optimal Coherence Value: ", max_cv)

### Final Model Training

lda_model = gensim.models.LdaModel(corpus=corpus,
                                      id2word=id2word,
                                      num_topics=num_topics,
                                      chunksize=100,
                                      passes=10,
                                      alpha=alpha,
                                      eta=beta)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics(num_topics=-1, num_words=20))
doc_lda = lda_model[corpus]

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

100%|██████████| 1/1 [4:34:00<00:00, 16440.95s/it]


alpha:  auto
beta:  1.81
Number of topics:  8
Optimal Coherence Value:  0.45127245101691355
[(0,
  '0.008*"hear" + 0.007*"king" + 0.006*"pass" + 0.006*"heart" + 0.005*"call" + '
  '0.005*"speak" + 0.005*"city" + 0.005*"father" + 0.005*"servant" + '
  '0.005*"earth" + 0.005*"day" + 0.004*"place" + 0.004*"word" + 0.004*"set" + '
  '0.004*"time" + 0.004*"eye" + 0.004*"brother" + 0.004*"deliver" + '
  '0.004*"stand" + 0.004*"bring"'),
 (1,
  '0.000*"hear" + 0.000*"city" + 0.000*"call" + 0.000*"speak" + 0.000*"pass" + '
  '0.000*"bring" + 0.000*"king" + 0.000*"day" + 0.000*"father" + 0.000*"place" '
  '+ 0.000*"servant" + 0.000*"eat" + 0.000*"sin" + 0.000*"priest" + '
  '0.000*"heart" + 0.000*"stand" + 0.000*"blood" + 0.000*"water" + '
  '0.000*"word" + 0.000*"year"'),
 (2,
  '0.009*"priest" + 0.009*"offer" + 0.006*"family" + 0.005*"levite" + '
  '0.005*"city" + 0.005*"bring" + 0.005*"number" + 0.005*"year" + '
  '0.004*"altar" + 0.004*"father" + 0.004*"eat" + 0.004*"unclean" + '
  '0.004*"

C:\Users\15138\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.197191  0.029167       1        1  82.815687
2      0.082251 -0.061658       2        1  13.964441
4     -0.029299  0.044630       3        1   3.126280
3     -0.045552  0.000174       4        1   0.089903
5     -0.048609 -0.001741       5        1   0.001139
7     -0.049783 -0.003165       6        1   0.001046
6     -0.052732 -0.003583       7        1   0.000758
1     -0.053467 -0.003824       8        1   0.000747, topic_info=                  Term         Freq        Total Category  logprob  loglift
878             priest   501.000000   501.000000  Default  30.0000  30.0000
636               king   886.000000   886.000000  Default  29.0000  29.0000
549               hear  1029.000000  1029.000000  Default  28.0000  28.0000
190               city   754.000000   754.000000  Default  27.0000  27.0000
137              bring   604.000000   604.000000  Default  26.0000  26.0000
551              heart   737.000000   737.000000  Default  25.0000  25.0000
815               pass   806.000000   806.000000  Default  24.0000  24.0000
409             father   735.000000   735.000000  Default  23.0000  23.0000
697               love   347.000000   347.000000  Default  22.0000  22.0000
159               call   717.000000   717.000000  Default  21.0000  21.0000
341              earth   646.000000   646.000000  Default  20.0000  20.0000
1074             speak   724.000000   724.000000  Default  19.0000  19.0000
792              offer   302.000000   302.000000  Default  18.0000  18.0000
346                eat   585.000000   585.000000  Default  17.0000  17.0000
387               evil   502.000000   502.000000  Default  16.0000  16.0000
1292              word   583.000000   583.000000  Default  15.0000  15.0000
998            servant   673.000000   673.000000  Default  14.0000  14.0000
255                day   654.000000   654.000000  Default  13.0000  13.0000
140            brother   568.000000   568.000000  Default  12.0000  12.0000
1307              year   473.000000   473.000000  Default  11.0000  11.0000
36              answer   489.000000   489.000000  Default  10.0000  10.0000
847              place   619.000000   619.000000  Default   9.0000   9.0000
886            prophet   308.000000   308.000000  Default   8.0000   8.0000
1953             write   377.000000   377.000000  Default   7.0000   7.0000
1293              work   548.000000   548.000000  Default   6.0000   6.0000
1246             water   490.000000   490.000000  Default   5.0000   5.0000
1268            wicked   299.000000   299.000000  Default   4.0000   4.0000
393                eye   534.000000   534.000000  Default   3.0000   3.0000
413               fear   433.000000   433.000000  Default   2.0000   2.0000
909            receive   284.000000   284.000000  Default   1.0000   1.0000
886            prophet   305.456970   308.144327   Topic1  -6.0292   0.1798
5636          disciple   225.703160   227.941419   Topic1  -6.3318   0.1787
1268            wicked   296.004355   299.351991   Topic1  -6.0606   0.1773
47                 art   321.268039   326.192644   Topic1  -5.9787   0.1733
943          righteous   201.696024   204.801566   Topic1  -6.4443   0.1733
387               evil   493.913514   502.076166   Topic1  -5.5487   0.1722
253           darkness   130.008295   132.294319   Topic1  -6.8834   0.1711
975               save   198.886104   202.427504   Topic1  -6.4583   0.1709
1942            wisdom   203.211464   207.054295   Topic1  -6.4368   0.1698
4469              shew   153.266123   156.166966   Topic1  -6.7188   0.1698
454             forget    93.598765    95.391718   Topic1  -7.2120   0.1696
2967            vanity    88.368741    90.084521   Topic1  -7.2695   0.1693
1193           trouble   132.794301   135.395395   Topic1  -6.8622   0.1692
36              answer   480.068862   489.498850   Topic1  -5.5771   0.16

### Average Model Parameters

In [15]:
# Topics range
min_topics = num_topics - 5
max_topics = num_topics + 5
if min_topics <= 2:
    min_topics = 2
    max_topics = 10
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

# List of alpha parameters
alpha_lst = ['auto']

# List of beta parameters
beta_lst = list(np.arange(beta - 0.9, beta + 1.3, 0.3))

avg_beta = 0.0
avg_num_topics = 0
avg_cv = 0.0
num_runs = 50

for run in range(num_runs):
    # Call to the method above
    alpha, beta, num_topics, max_cv = _get_best_num_topics(topics_range, alpha_lst, beta_lst, corpus)
    avg_beta += beta
    avg_num_topics += num_topics
    avg_cv += max_cv
    
avg_beta /= num_runs
avg_num_topics /= num_runs
avg_cv /= num_runs

print("Avg beta: ", avg_beta)
print("Avg Number of topics: ", avg_num_topics)
print("Avg Optimal Coherence Value: ", avg_cv)

100%|██████████| 1/1 [10:29<00:00, 629.69s/it]

Avg beta:  2.0399999999999996
Avg Number of topics:  7.68
Avg Optimal Coherence Value:  0.6255105704402016


### Run Example Models Using Average Parameters

In [16]:
for run in range(1):
    print("-------------------------------------------Run Number " + str(run) + "---------------------------------------------------------")
    print()
    ### Final Model Training

    lda_model = gensim.models.LdaModel(corpus=corpus,
                                          id2word=id2word,
                                          num_topics=8,
                                          chunksize=100,
                                          passes=10,
                                          alpha='auto',
                                          eta=2.04)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    print("Coherence: " + str(coherence_model_lda.get_coherence()))
    print()

    # Print the Keyword in the 20 topics
    pprint(lda_model.print_topics(num_topics=-1, num_words=20))
    doc_lda = lda_model[corpus]
    
    # Visualize the topics
    pyLDAvis.enable_notebook(lda_model)
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word, mds='mmds')
    pyLDAvis.save_html(LDAvis_prepared, "LDAvis_KJV_B_" + str(run) + ".html")
    print()
    print()

-------------------------------------------Run Number 0---------------------------------------------------------

Coherence: 0.33203607463213836

[(0,
  '0.000*"king" + 0.000*"speak" + 0.000*"hear" + 0.000*"pass" + 0.000*"call" + '
  '0.000*"turn" + 0.000*"father" + 0.000*"priest" + 0.000*"day" + 0.000*"eat" '
  '+ 0.000*"earth" + 0.000*"fall" + 0.000*"place" + 0.000*"water" + '
  '0.000*"city" + 0.000*"sin" + 0.000*"deliver" + 0.000*"work" + 0.000*"time" '
  '+ 0.000*"servant"'),
 (1,
  '0.000*"pass" + 0.000*"city" + 0.000*"host" + 0.000*"king" + 0.000*"hear" + '
  '0.000*"earth" + 0.000*"water" + 0.000*"heart" + 0.000*"eye" + 0.000*"enemy" '
  '+ 0.000*"stand" + 0.000*"turn" + 0.000*"eat" + 0.000*"dwell" + '
  '0.000*"smite" + 0.000*"bring" + 0.000*"nation" + 0.000*"set" + '
  '0.000*"servant" + 0.000*"cut"'),
 (2,
  '0.000*"hear" + 0.000*"speak" + 0.000*"set" + 0.000*"bring" + 0.000*"side" + '
  '0.000*"sword" + 0.000*"place" + 0.000*"pass" + 0.000*"city" + 0.000*"face" '
  '+ 0.000

C:\Users\15138\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \


## Chapters

### Preprocessing

In [17]:
# Create list of chapter documents and put them in a dataframe
chapterList = []
indexList = []
for chapter in kjv.groupby(['b','c'])['t']:
    chapterList.append(' '.join(chapter[1].tolist()))
    indexList.append(chapter[0])
chapters = pd.DataFrame({'b,c': indexList, 't': chapterList})

In [18]:
# Remove punctuation
chapters['t_processed'] = chapters['t'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
chapters['t_processed'] = chapters['t_processed'].map(lambda x: x.lower())

#### Tokenize words and further clean-up text

data = chapters.t_processed.values.tolist()
data_words = list(sent_to_words(data))

#### Creating Bigram and Trigram Models

# Build the bigram and trigram models

bigram = gensim.models.Phrases(data_words, min_count=3, threshold=5)
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#### Remove Stopwords, Make Bigrams and Lemmatize

#list of stopwords
with open('stop_words_lst.txt') as f:
    standard_stop_words = f.read().splitlines()
with open('bible_stop_words_lst.txt') as f:
    bible_stop_words = f.read().splitlines()
stop_words = standard_stop_words + bible_stop_words

### Call the functions above

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_lg", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

### Data transformation: Corpus and Dictionary

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

<>:2: DeprecationWarning: invalid escape sequence \.
<>:2: DeprecationWarning: invalid escape sequence \.
C:\Users\15138\AppData\Local\Temp/ipykernel_8720/92508175.py:2: DeprecationWarning: invalid escape sequence \.
  chapters['t_processed'] = chapters['t'].map(lambda x: re.sub('[,\.!?]', '', x))


In [12]:
# Topics range
min_topics = 2
max_topics = 66
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

# List of alpha parameters
alpha_lst = ['auto']

# List of beta parameters
beta_lst = list(np.arange(0.01, 3, 0.1))

# Call to the method above
alpha, beta, num_topics, max_cv = _get_best_num_topics(topics_range, alpha_lst, beta_lst, corpus)

print("alpha: ", alpha)
print("beta: ", beta)
print("Number of topics: ", num_topics)
print("Optimal Coherence Value: ", max_cv)

### Final Model Training

lda_model = gensim.models.LdaModel(corpus=corpus,
                                      id2word=id2word,
                                      num_topics=num_topics,
                                      chunksize=100,
                                      passes=10,
                                      alpha=alpha,
                                      eta=beta)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics(num_topics=-1, num_words=20))
doc_lda = lda_model[corpus]

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word, mds='mmds')
LDAvis_prepared

100%|██████████| 1/1 [5:48:10<00:00, 20890.77s/it]


alpha:  auto
beta:  2.61
Number of topics:  22
Optimal Coherence Value:  0.8059847299146714
[(0,
  '0.000*"sail" + 0.000*"anchor" + 0.000*"boat" + 0.000*"fathom" + 0.000*"sir" '
  '+ 0.000*"swim" + 0.000*"loss" + 0.000*"deem" + 0.000*"foreship" + '
  '0.000*"dangerous" + 0.000*"aground" + 0.000*"adramyttium" + 0.000*"cnidus" '
  '+ 0.000*"forepart" + 0.000*"lading" + 0.000*"scarce" + 0.000*"crete" + '
  '0.000*"mainsail" + 0.000*"abstinence" + 0.000*"hoise"'),
 (1,
  '0.000*"greet" + 0.000*"amplia" + 0.000*"salute" + 0.000*"chedorlaomer" + '
  '0.000*"apelle" + 0.000*"aristobulus" + 0.000*"herodion" + 0.000*"assist" + '
  '0.000*"narcissus" + 0.000*"olympas" + 0.000*"phlegon" + 0.000*"urbane" + '
  '0.000*"herma" + 0.000*"holy_kiss" + 0.000*"watcheth" + '
  '0.000*"wicked_device" + 0.000*"disquietness" + 0.000*"payeth" + '
  '0.000*"transgressor" + 0.000*"evildoer"'),
 (2,
  '0.000*"envious" + 0.000*"mourneth" + 0.000*"abject" + 0.000*"gnash" + '
  '0.000*"conversation" + 0.000*"slippe

C:\Users\15138\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.010602  0.229720       1        1  92.796923
10    -0.027748  0.001701       2        1   6.592370
14     0.000115 -0.011522       3        1   0.370596
7      0.000860 -0.011567       4        1   0.047037
16     0.000815 -0.011848       5        1   0.031959
12     0.001091 -0.011635       6        1   0.026968
6      0.001002 -0.011737       7        1   0.024744
3      0.001123 -0.011336       8        1   0.020755
19     0.000619 -0.011694       9        1   0.020234
15     0.000877 -0.011547      10        1   0.005647
13     0.000868 -0.011530      11        1   0.005323
0      0.000864 -0.011549      12        1   0.005296
5      0.000926 -0.011545      13        1   0.005293
1      0.000898 -0.011539      14        1   0.005278
17     0.000904 -0.011538      15        1   0.005271
8      0.000906 -0.011537      16        1   0.005239
21     0.000894 -0.011539      17        1   0.005216
9      0.000906 -0.011537      18        1   0.005202
18     0.000866 -0.011557      19        1   0.005187
2      0.000859 -0.011566      20        1   0.005177
20     0.000878 -0.011548      21        1   0.005150
11     0.000877 -0.011549      22        1   0.005139, topic_info=                 Term         Freq        Total Category  logprob  loglift
191              hear  1132.000000  1132.000000  Default  30.0000  30.0000
417             speak   821.000000   821.000000  Default  29.0000  29.0000
6                call   744.000000   744.000000  Default  28.0000  28.0000
514              word   714.000000   714.000000  Default  27.0000  27.0000
177           brother   686.000000   686.000000  Default  26.0000  26.0000
253             heart   689.000000   689.000000  Default  25.0000  25.0000
18              earth   642.000000   642.000000  Default  24.0000  24.0000
88             father   647.000000   647.000000  Default  23.0000  23.0000
458              king   627.000000   627.000000  Default  22.0000  22.0000
204              pass   615.000000   615.000000  Default  21.0000  21.0000
209               sin   611.000000   611.000000  Default  20.0000  20.0000
218              time   595.000000   595.000000  Default  19.0000  19.0000
742              love   556.000000   556.000000  Default  18.0000  18.0000
179              city   591.000000   591.000000  Default  17.0000  17.0000
206           receive   535.000000   535.000000  Default  16.0000  16.0000
13                day   571.000000   571.000000  Default  15.0000  15.0000
87               fall   521.000000   521.000000  Default  14.0000  14.0000
1568            write   521.000000   521.000000  Default  13.0000  13.0000
572             stand   541.000000   541.000000  Default  12.0000  12.0000
86               evil   504.000000   504.000000  Default  11.0000  11.0000
352           servant   483.000000   483.000000  Default  10.0000  10.0000
2934            faith   470.000000   470.000000  Default   9.0000   9.0000
124              work   647.000000   647.000000  Default   8.0000   8.0000
653              cast   505.000000   505.000000  Default   7.0000   7.0000
141               eye   471.000000   471.000000  Default   6.0000   6.0000
83                eat   473.000000   473.000000  Default   5.0000   5.0000
5               bring   493.000000   493.000000  Default   4.0000   4.0000
538            answer   447.000000   447.000000  Default   3.0000   3.0000
594               law   452.000000   452.000000  Default   2.0000   2.0000
338           deliver   440.000000   440.000000  Default   1.0000   1.0000
191              hear  1131.021603  1132.328960   Topic1  -4.8348   0.0736
417             speak   820.482946   821.889371   Topic1  -5.1558   0.0730
6                call   743.198736   744.514435   Topic1  -5.2547   0.0730
514              word   713.546829   714.817414   Topic1  -5.2955   0.0730
177           brother   685.080852  

### Average Model Parameters

In [13]:
# Topics range
min_topics = num_topics - 5
max_topics = num_topics + 5
if min_topics <= 2:
    min_topics = 2
    max_topics = 10
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

# List of alpha parameters
alpha_lst = ['auto']

# List of beta parameters
beta_lst = list(np.arange(beta - 0.9, beta + 1.3, 0.3))

avg_beta = 0.0
avg_num_topics = 0
avg_cv = 0.0
num_runs = 50

for run in range(num_runs):
    # Call to the method above
    alpha, beta, num_topics, max_cv = _get_best_num_topics(topics_range, alpha_lst, beta_lst, corpus)
    avg_beta += beta
    avg_num_topics += num_topics
    avg_cv += max_cv
    
avg_beta /= num_runs
avg_num_topics /= num_runs
avg_cv /= num_runs

print("Avg beta: ", avg_beta)
print("Avg Number of topics: ", avg_num_topics)
print("Avg Optimal Coherence Value: ", avg_cv)

100%|██████████| 1/1 [10:02<00:00, 602.60s/it]

Avg beta:  3.191999999999999
Avg Number of topics:  18.8
Avg Optimal Coherence Value:  0.804068649418893


### Run Example Models Using Average Parameters

In [19]:
for run in range(1):
    print("-------------------------------------------Run Number " + str(run) + "---------------------------------------------------------")
    print()
    ### Final Model Training

    lda_model = gensim.models.LdaModel(corpus=corpus,
                                          id2word=id2word,
                                          num_topics=19,
                                          chunksize=100,
                                          passes=10,
                                          alpha='auto',
                                          eta=3.19)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    print("Coherence: " + str(coherence_model_lda.get_coherence()))
    print()

    # Print the Keyword in the 20 topics
    pprint(lda_model.print_topics(num_topics=-1, num_words=20))
    doc_lda = lda_model[corpus]

    # Get dominant topic
    df_topic_sents_keywords = format_topics_sentences(lda_model, corpus, data_lemmatized)
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    display(df_dominant_topic)

    # Visualize the topics
    pyLDAvis.enable_notebook()
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word, mds='mmds')
    pyLDAvis.save_html(LDAvis_prepared, "LDAvis_KJV_C_" + str(run) + ".html")
    print()
    print()

-------------------------------------------Run Number 0---------------------------------------------------------

Coherence: 0.6538970168261462

[(0,
  '0.008*"hear" + 0.006*"speak" + 0.005*"call" + 0.005*"word" + 0.005*"heart" '
  '+ 0.005*"brother" + 0.004*"work" + 0.004*"father" + 0.004*"earth" + '
  '0.004*"king" + 0.004*"pass" + 0.004*"sin" + 0.004*"time" + 0.004*"city" + '
  '0.004*"day" + 0.004*"love" + 0.004*"stand" + 0.004*"receive" + '
  '0.004*"write" + 0.004*"fall"'),
 (1,
  '0.000*"envious" + 0.000*"mourneth" + 0.000*"abject" + 0.000*"gnash" + '
  '0.000*"conversation" + 0.000*"slippeth" + 0.000*"evildoer" + '
  '0.000*"patiently" + 0.000*"payeth" + 0.000*"transgressor" + '
  '0.000*"watcheth" + 0.000*"wicked_device" + 0.000*"aloof" + '
  '0.000*"disquietness" + 0.000*"lover" + 0.000*"panteth" + 0.000*"presseth" + '
  '0.000*"borroweth" + 0.000*"deliverest" + 0.000*"slippery"'),
 (2,
  '0.000*"envious" + 0.000*"mourneth" + 0.000*"abject" + 0.000*"gnash" + '
  '0.000*"conve

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.0,0.9992,"hear, speak, call, word, heart, brother, work,...","[god_create, earth, earth, form, void, darknes..."
1,1,0.0,0.9985,"hear, speak, call, word, heart, brother, work,...","[finish, host, seventh, end, work, rest, seven..."
2,2,0.0,0.9988,"hear, speak, call, word, heart, brother, work,...","[serpent, subtil, beast, field, eat, tree, gar..."
3,3,0.0,0.9986,"hear, speak, call, word, heart, brother, work,...","[know, wife, conceive, tiller, ground, process..."
4,4,0.0,0.9984,"hear, speak, call, word, heart, brother, work,...","[book, generation, god_create, likeness, male,..."
5,5,0.0,0.9984,"hear, speak, call, word, heart, brother, work,...","[pass, begin, multiply, face, earth, daughter,..."
6,6,0.0,0.9988,"hear, speak, call, word, heart, brother, work,...","[generation, clean, beast, seven, male, female..."
7,7,0.0,0.9988,"hear, speak, call, word, heart, brother, work,...","[remember, cattle, wind, pass, earth, water, a..."
8,8,0.0,0.9987,"hear, speak, call, word, heart, brother, work,...","[bless, fruitful, multiply, replenish, earth, ..."
9,9,0.0,0.8243,"hear, speak, call, word, heart, brother, work,...","[generation, bear, dodanim, isle, gentile, div..."


C:\Users\15138\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \
